# parcels regions

runs parcels on existing netcdf files

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import utils
from parcels_utils import HFRGrid, read_netcdf_info
from parcels_sim import ParcelsSimulation
from parcels_analysis import add_feature_set_to_result
from plot_features import BuoyPathFeature
from gapfilling import Gapfiller

## configuration

change the contents of `configs` for the simulation configuration you want

In [ ]:
configs = [
#     "plume_track.json",
#     "tijuana_now.json",
#     "buoy_track_704_02.json",
    "hunington_spill_elly.json",
    "hunington_spill_elbow.json",
    "hunington_spill_leg.json"
]

loaded_configs = [utils.load_config(utils.PARCELS_CONFIGS_DIR / path) for path in configs]
grids = []
for cfg in loaded_configs:
    ds = read_netcdf_info(cfg["netcdf_path"])
    gapfiller = Gapfiller.load_from_config(*cfg["netcdf_path"]["gapfill_steps"])
    ds = gapfiller.execute(HFRGrid(ds))
    bound_cond = cfg["parcels_config"].get("boundary", None)
    if bound_cond is None:
        grids.append(HFRGrid(ds))
    elif bound_cond.lower() in ("free", "freeslip"):
        grids.append(HFRGrid(ds, fs_kwargs={"interp_method": {"U": "freeslip", "V": "freeslip"}}))
    elif bound_cond.lower() in ("partial", "partialslip"):
        grids.append(HFRGrid(ds, fs_kwargs={"interp_method": {"U": "partialslip", "V": "partialslip"}}))
    else:
        grids.append(HFRGrid(ds))
        
sims = []
for i in range(len(loaded_configs)):
    sim = ParcelsSimulation(loaded_configs[i]["name"], grids[i], loaded_configs[i]["parcels_config"])
    sims.append(sim)

## Animated gif stuff and particle simulation

### ParticleSet and spawn point setup

note about interpolation methods: only `linear` works if you want to use the FieldSet in a ParticleSet.

In [ ]:
[sim.pset.show(field="vector") for sim in sims]

### simulation execution

In [ ]:
for sim in sims:
    sim.execute()

### plot and gif generation

gif generation requires [magick](https://imagemagick.org/index.php)

if it doesn't work just comment out the gif generation line

the gifs will be saved `snapshots/`

In [ ]:
import numpy as np

def handle_postprocessing(result, postprocess_cfg):
    if "coastline" in postprocess_cfg and postprocess_cfg["coastline"] not in empty:
        if postprocess_cfg["coastline"]["path"] not in (None, ""):
            # coastline stuff, use to delete particles that collide with a coastline
            lats, lons = utils.load_geo_points(**postprocess_cfg["coastline"])
            result.add_coastline(lats, lons)
            result.process_coastline_collisions()
            print("processed collisions")
    if "buoy" in postprocess_cfg and postprocess_cfg["buoy"] not in empty:
        if postprocess_cfg["buoy"]["path"] not in (None, ""):
            result.add_plot_feature(
                BuoyPathFeature.from_csv(
                    postprocess_cfg["buoy"]["path"], backstep_delta=np.timedelta64(1, "h"),
                    backstep_count=12
                ), "buoy"
            )
            result.write_feature_dists(["buoy"])
            print("processed buoy distances")
    result.write_data(override=True)

empty = (None, {}, "", [])
for i, sim in enumerate(sims):
    cfg = loaded_configs[i]
    postprocess_cfg = cfg["postprocess_config"]
    handle_postprocessing(sim.parcels_result, postprocess_cfg)
    sim.parcels_result.write_data(override=True)
    if cfg["save_snapshots"]:
        plotting_cfg = cfg["plotting_config"]
        if "plot_feature_set" in plotting_cfg and plotting_cfg["plot_feature_set"] not in (None, ""):
            add_feature_set_to_result(sim.parcels_result, plotting_cfg["plot_feature_set"])
        sim.parcels_result.generate_all_plots(
            os.path.join(plotting_cfg["save_dir_snapshots"], sim.name),
            domain=plotting_cfg["shown_domain"],
            land=plotting_cfg["draw_coasts"],
            figsize=(13, 8)
        )
        # comment out the generate_gif line if it doesn't work
        print(sim.parcels_result.generate_gif(os.path.join(plotting_cfg["save_dir_snapshots"], f"{sim.name}.gif")))